# NLP Disaster Tweets Project

Our goal is to solve a Natural Language Processing (NLP) problem of predicting whether tweets describe real disasters. 

The challenge of this problem is that people may use words in their tweets that describe a disaster, but are not describing a real disaster. To solve this problem, we use the DistilBert model, which is a pre-trained deep learning model that understands the contextual meaning of text. 

We first preprocessed the text, including conversion to lowercase, word splitting, deactivation, and stemming extraction, and then trained and predicted with the DistilBert model. Finally, we use the confusion matrix to evaluate the performance of the model and submit the predictions to the competition platform. We hope this brief will help you understand our problem and solution.

In [ ]:
!pip install keras-core --upgrade
!pip install -q keras-nlp --upgrade
import os
import numpy as np
import pandas as pd
import tensorflow as tf
import keras_core as keras
import keras_nlp
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

The selected backend is TensorFlow, while pyTorch can also be selected.

In [ ]:
os.environ['KERAS_BACKEND'] = 'tensorflow'

print("TensorFlow version:", tf.__version__)
print("KerasNLP version:", keras_nlp.__version__)

## EDA

### Data Importing

In [ ]:
df_train = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
df_test = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

print('Training Set Shape = {}'.format(df_train.shape))
print('Training Set Memory Usage = {:.2f} MB'.format(df_train.memory_usage().sum() / 1024**2))
print('Test Set Shape = {}'.format(df_test.shape))
print('Test Set Memory Usage = {:.2f} MB'.format(df_test.memory_usage().sum() / 1024**2))

### Data Preprocessing

Creating instances of stop words and stemming extractors

In [ ]:
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

Pre-processing procedures:

- Changing letters to lower case
- remove stop words

In [ ]:
def preprocess_text(text):
    text = text.lower()
    words = text.split()
    words = [ps.stem(word) for word in words if word not in stop_words]
    text = ' '.join(words)
    return text

In [ ]:
df_train["text"] = df_train["text"].apply(preprocess_text)
df_test["text"] = df_test["text"].apply(preprocess_text)

df_train["length"] = df_train["text"].apply(lambda x : len(x))
df_test["length"] = df_test["text"].apply(lambda x : len(x))

print("Train Length Stat")
print(df_train["length"].describe())
print("Test Length Stat")
print(df_test["length"].describe())

### Data Visualization

Histogram of Tweet Length

In [ ]:
import matplotlib.pyplot as plt

# Histogram of Tweet Length
plt.figure(figsize=(10, 6))
plt.hist(df_train['length'], bins=30, alpha=0.7, color='dodgerblue')
plt.title('Histogram of Tweet Length')
plt.xlabel('Length')
plt.ylabel('Frequency')
plt.show()

Pie Chart of Disaster vs Non-disaster Tweets

In [ ]:
plt.figure(figsize=(6, 6))
df_train['target'].value_counts().plot(kind='pie', autopct='%1.1f%%', colors=['skyblue', 'gold'])
plt.title('Pie Chart of Disaster vs Non-disaster Tweets')
plt.ylabel('')
plt.show()

Average Tweet Length of Disaster vs Non-disaster

In [ ]:
avg_len_disaster = df_train[df_train['target'] == 1]['length'].mean()
avg_len_non_disaster = df_train[df_train['target'] == 0]['length'].mean()

plt.figure(figsize=(6, 6))
plt.bar(['Disaster', 'Non-disaster'], [avg_len_disaster, avg_len_non_disaster], color='seagreen')
plt.title('Average Tweet Length of Disaster vs Non-disaster')
plt.xlabel('Type')
plt.ylabel('Average Length')
plt.show()

## Model Params

Will use the DistilBert model.

In [ ]:
BATCH_SIZE = 64
NUM_TRAINING_EXAMPLES = df_train.shape[0]
TRAIN_SPLIT = 0.8
VAL_SPLIT = 0.2
STEPS_PER_EPOCH = int(NUM_TRAINING_EXAMPLES)*TRAIN_SPLIT // BATCH_SIZE
EPOCHS = 3
AUTO = tf.data.experimental.AUTOTUNE

## Model Training



DistilBERT is a lightweight BERT that retains most of the performance of BERT, but with a model half the size of BERT and faster training and prediction. Here are some key features of DistilBERT:

Transformer Architecture: DistilBERT is based on the Transformer architecture, a deep learning model architecture that is particularly well suited for processing sequential data, such as text.The key feature of the Transformer architecture is the self-attention mechanism, which allows the model to process each word taking into account all other words in the text.
Pre-training: DistilBERT is a pre-trained model, meaning that it has been pre-trained on large-scale textual data to learn a rich linguistic representation. This enables DistilBERT to understand the contextual meaning of the text and thus efficiently handle various NLP tasks.
Distillation: The process of creating DistilBERT involves a technique called "knowledge distillation". In knowledge distillation, a large model (in this case BERT) is used as a teacher model to instruct a small model (in this case DistilBERT). The small model is trained to mimic the behavior of the teacher model, which allows the small model to learn the knowledge of the teacher model while maintaining a smaller model size and faster training speed.

**MODEL ARCHITECTURE**: The architecture of DistilBERT is very similar to that of BERT in that both include multiple layers of Transformer encoders. Each layer includes a self-attention mechanism and a feed-forward neural network. However, DistilBERT omits some layers in BERT, such as token type embeddings and pooling.


Divide the data into a training set and a validation set.

In [ ]:
from sklearn.model_selection import train_test_split

X = df_train["text"]
y = df_train["target"]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=VAL_SPLIT, random_state=42)

X_test = df_test["text"]

We use Keras Core, the multi-backend version of Keras for model building.

Load a DistilBERT model, make the sequences shorter through preprocessor, and finally get a pretrained classifier for final compile and fit process.

In [ ]:
preset= "distil_bert_base_en_uncased"
preprocessor = keras_nlp.models.DistilBertPreprocessor.from_preset(preset, sequence_length=160, name="preprocessor_4_tweets")
classifier = keras_nlp.models.DistilBertClassifier.from_preset(preset,preprocessor = preprocessor,num_classes=2)

classifier.summary()

classifier.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(1e-5),
    metrics= ["accuracy"]  
)

history = classifier.fit(x=X_train,
                         y=y_train,
                         batch_size=BATCH_SIZE,
                         epochs=EPOCHS, 
                         validation_data=(X_val, y_val))

### Evaluation

We can have a more direct visualization about the model quality.

In [ ]:
def displayConfusionMatrix(y_true, y_pred, dataset):
    disp = ConfusionMatrixDisplay.from_predictions(
        y_true,
        np.argmax(y_pred, axis=1),
        display_labels=["Not Disaster","Disaster"],
        cmap=plt.cm.Blues
    )

    tn, fp, fn, tp = confusion_matrix(y_true, np.argmax(y_pred, axis=1)).ravel()
    f1_score = tp / (tp+((fn+fp)/2))

    disp.ax_.set_title("Confusion Matrix on " + dataset + " Dataset -- F1 Score: " + str(f1_score.round(2)))


In [ ]:
y_pred_train = classifier.predict(X_train)

displayConfusionMatrix(y_train, y_pred_train, "Training")

In [ ]:
y_pred_val = classifier.predict(X_val)

displayConfusionMatrix(y_val, y_pred_val, "Validation")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

y_pred_train2 = np.argmax(classifier.predict(X_train), axis=1)

accuracy_train = accuracy_score(y_train, y_pred_train2)
precision_train = precision_score(y_train, y_pred_train2)
recall_train = recall_score(y_train, y_pred_train2)
f1_train = f1_score(y_train, y_pred_train2)

print('Training Accuracy: {:.2f}'.format(accuracy_train))
print('Training Precision: {:.2f}'.format(precision_train))
print('Training Recall: {:.2f}'.format(recall_train))
print('Training F1 Score: {:.2f}'.format(f1_train))

y_pred_val2 = np.argmax(classifier.predict(X_val), axis=1)

accuracy_val = accuracy_score(y_val, y_pred_val2)
precision_val = precision_score(y_val, y_pred_val2)
recall_val = recall_score(y_val, y_pred_val2)
f1_val = f1_score(y_val, y_pred_val2)

print('Validation Accuracy: {:.2f}'.format(accuracy_val))
print('Validation Precision: {:.2f}'.format(precision_val))
print('Validation Recall: {:.2f}'.format(recall_val))
print('Validation F1 Score: {:.2f}'.format(f1_val))


metrics_train = [accuracy_train, precision_train, recall_train, f1_train]
metrics_val = [accuracy_val, precision_val, recall_val, f1_val]

labels = ['Accuracy', 'Precision', 'Recall', 'F1 Score']
x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots()
rects1 = ax.bar(x - width/2, metrics_train, width, label='Train')
rects2 = ax.bar(x + width/2, metrics_val, width, label='Validation')

ax.set_ylabel('Scores')
ax.set_title('Scores by group and gender')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

fig.tight_layout()

plt.show()


## Prediction Output

And print the output to file.

In [ ]:
sample_submission = pd.read_csv("/kaggle/input/nlp-getting-started/sample_submission.csv")

sample_submission["target"] = np.argmax(classifier.predict(X_test), axis=1)

sample_submission.to_csv("submission.csv", index=False)

## Conclusion

In this project, we successfully solved a natural language processing (NLP) problem of predicting whether tweets describe real disasters. 

We first performed a series of preprocessing steps on the tweets, including conversion to lowercase, word splitting, deactivation, and stemming extraction, in order to allow the model to better understand the meaning of the text. 

We then trained and predicted using the DistilBert model, a powerful pre-trained deep learning model that understands the contextual meaning of the text to effectively distinguish between tweets describing real disasters and those that are not. 

During the model training and prediction process, we also use a confusion matrix to evaluate the model's performance and ensure that our model can accurately predict real disaster tweets. 

Finally, we completed the project by submitting our predictions to the competition platform. Overall, this project demonstrates how deep learning techniques can be used to solve real-world NLP problems and provides a valuable reference for future related research and applications.

The selected backend is TensorFlow, while pyTorch can also be selected.